In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
from glob import glob
import re
import ast
import numpy as np 
import pandas as pd
from PIL import Image, ImageDraw 
from tqdm import tqdm
from dask import bag

import tensorflow as tf
from tensorflow import keras
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNet,MobileNetV2
from tensorflow.keras.applications.mobilenet import preprocess_input



In [0]:
class_paths =glob('/content/gdrive/My Drive/input/train/*.csv')
numstonames = {i: v[14:-4].replace(" ", "_") for i, v in enumerate(class_paths)}
for i in numstonames:
  numstonames[i]= numstonames[i].replace("e/My_Drive/input/train/","")
print(numstonames)

namestonums = {v:k for k,v in numstonames.items()}

num_classes = 30
imheight, imwidth = 64, 64  
ims_per_class = 12000

def draw_it(strokes):
    image = Image.new("P", (64,64), color=255)
    image_draw = ImageDraw.Draw(image)
    for stroke in ast.literal_eval(strokes):
        for i in range(len(stroke[0])-1):
            image_draw.line([stroke[0][i], 
                             stroke[1][i],
                             stroke[0][i+1], 
                             stroke[1][i+1]],
                            fill=0, width=5)
    image = image.resize((imheight, imwidth))
    return np.array(image)/255.

{0: 'banana', 1: 'bandage', 2: 'bear', 3: 'bed', 4: 'bee', 5: 'belt', 6: 'cactus', 7: 'cake', 8: 'coffee_cup', 9: 'door', 10: 'fork', 11: 'giraffe', 12: 'hand', 13: 'key', 14: 'laptop', 15: 'light_bulb', 16: 'marker', 17: 'paintbrush', 18: 'popsicle', 19: 'raccoon', 20: 'rain', 21: 'roller_coaster', 22: 'snail', 23: 'spoon', 24: 'swan', 25: 'The_Great_Wall_of_China', 26: 'toaster', 27: 'train', 28: 'whale', 29: 'wine_bottle'}


In [0]:
model = tf.keras.models.load_model('/content/gdrive/My_Drive/input/train/19.model')

model.compile(optimizer = Adam(lr = 0.001), loss = 'categorical_crossentropy',
              metrics = ["accuracy"])
model.summary()

OSError: ignored

In [0]:
test = pd.read_csv('/content/gdrive/My_Drive/input/test.csv')[['drawing']]
imagebag = bag.from_sequence(test.drawing.values).map(draw_it)
testarray = np.array(imagebag.compute())
testarray = np.reshape(testarray, (testarray.shape[0], imheight, imwidth, 1))
testpreds = model.predict(testarray, verbose=0)
ttvs = np.argsort(-testpreds)

numstonames = {i: v[14:-4].replace(" ", "_") for i, v in enumerate(class_paths)}
for i in numstonames:
  numstonames[i]= numstonames[i].replace("/content/gdrive/My_Drive/input/train/","")

predict_label = [numstonames[ttvs[i][0]] for i in range(ttvs.shape[0])]

test['word'] = predict_label
test.to_csv('//content/gdrive/My_Drive/input/test_predict.csv')